In [1]:
import cv2
import mediapipe as mp
import tensorflow as tf
import time

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [5]:
image_path = './videos/vid.mp4'

image_path2 = './videos/inp.gif'
cap1 = cv2.VideoCapture(image_path)
cap2 = cv2.VideoCapture(image_path2)

In [6]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.6) as pose:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.6) as pose2:
        while cap1.isOpened() and cap2.isOpened():
            time.sleep(0.5)
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()

            

            if ret1 and ret2:
                frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
                frame1.flags.writeable= False
                frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
                frame2.flags.writeable= False

                results1 = pose.process(frame1)
                results2 = pose2.process(frame2)

                frame1 = cv2.cvtColor(frame1, cv2.COLOR_RGB2BGR)
                frame1.flags.writeable= True
                frame2 = cv2.cvtColor(frame2, cv2.COLOR_RGB2BGR)
                frame2.flags.writeable= True

                mp_drawing.draw_landmarks(
                frame1,
                results1.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

                mp_drawing.draw_landmarks(
                frame2,
                results2.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                cv2.imshow("Vid1", frame1)
                cv2.imshow("vid2", frame2)
            else:
                break


            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

cap1.release()
cap2.release()
cv2.destroyAllWindows()